In [ ]:
# Torch geometric modules
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

# Torch modules
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# SKlearn modules
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

# Ground modules
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool

# TropiGAT modules
import TropiGAT_graph
import TropiGAT_models

warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"
DF_info = pd.read_csv(f"{path_work}/train_nn/TropiGATv2.final_df_v2.tsv", sep = "\t" ,  header = 0)

DF_info = DF_info.drop_duplicates(subset = ["Protein_name"])

df_prophages = DF_info.drop_duplicates(subset = ["Phage"], keep = "first")
dico_prophage_info = {row["Phage"] : {"prophage_strain" : row["prophage_id"] , "ancestor" : row["Infected_ancestor"]} for _,row in df_prophages.iterrows()}

def get_filtered_prophages(prophage) :
    combinations = []
    to_exclude = set()
    to_keep = set()
    to_keep.add(prophage)
    df_prophage_group = DF_info[(DF_info["prophage_id"] == dico_prophage_info[prophage]["prophage_strain"]) & (DF_info["Infected_ancestor"] == dico_prophage_info[prophage]["ancestor"])]
    if len(df_prophage_group) == 1 :
        pass
    else :
        depo_set = set(df_prophage_group[df_prophage_group["Phage"] == prophage]["domain_seq"].values)
        for prophage_tmp in df_prophage_group["Phage"].unique().tolist() :
            if prophage_tmp != prophage :
                tmp_depo_set = set(df_prophage_group[df_prophage_group["Phage"] == prophage_tmp]["domain_seq"].values)
                if depo_set == tmp_depo_set :
                    to_exclude.add(prophage_tmp)
                else :
                    if tmp_depo_set not in combinations :
                        to_keep.add(prophage_tmp)
                        combinations.append(tmp_depo_set)
                    else :
                        to_exclude.add(prophage_tmp)
    return df_prophage_group , to_exclude , to_keep

good_prophages = set()
excluded_prophages = set()

for prophage, info_prophage in tqdm(dico_prophage_info.items()) :
    if prophage not in excluded_prophages and prophage not in good_prophages:
        _, excluded_members , kept_members = get_filtered_prophages(prophage)
        good_prophages.update(kept_members)
        excluded_prophages.update(excluded_members)

DF_info_lvl_0_filtered = DF_info[DF_info["Phage"].isin(good_prophages)]
DF_info_lvl_0_final = DF_info_lvl_0_filtered[~DF_info_lvl_0_filtered["KL_type_LCA"].str.contains("\\|")]

DF_info_lvl_0 = DF_info_lvl_0_final.copy()



# Log file :
path_ensemble = f"{path_work}/train_nn/ensemble_0702"

df_prophages = DF_info_lvl_0.drop_duplicates(subset = ["Phage"])
dico_prophage_count = dict(Counter(df_prophages["KL_type_LCA"]))

KLtypes = [kltype for kltype in dico_prophage_count if dico_prophage_count[kltype] >= 20]

metrics_df = open(f"{path_ensemble}/Metric_Report.0702.tsv").read().split("\n")
good_KLtypes  = [item.split("\t")[0] for  index,item in enumerate(metrics_df) if item if item.split("\t")[2] not in [0.0 , "***Issue***"]]
kltypes_to_repeat = [item.split("\t")[0] for  index,item in enumerate(metrics_df) if item if item.split("\t")[2] in [0.0 , "***Issue***"] if item.split("\t")[0] not in good_KLtypes]


# *****************************************************************************
# Make graphs :
graph_baseline , dico_prophage_kltype_associated = TropiGAT_graph.build_graph_baseline(DF_info_lvl_0)

# *****************************************************************************
def train_graph(KL_type) :
    for seed in range(1,6) :
        id_kl = f"{KL_type}__{seed}"
        if id_kl in kltypes_to_repeat :
            with open(f"{path_work}/train_nn/ensemble_0702_log_files/{KL_type}__{seed}__node_classification.0702.log" , "w") as log_outfile :
                n_prophage = dico_prophage_count[KL_type]
                graph_data_kltype = TropiGAT_graph.build_graph_masking_v2(graph_baseline , dico_prophage_kltype_associated, DF_info_lvl_0, KL_type, 5, 0.7, 0.2, 0.1, seed = seed)
                if n_prophage <= 125 : 
                    model = TropiGAT_models.TropiGAT_small_module(hidden_channels = 1280, heads = 1)
                    n = "small"
                else : 
                    model = TropiGAT_models.TropiGAT_big_module(hidden_channels = 1280, heads = 1)
                    n = "big"
                model(graph_data_kltype)
                optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001 , weight_decay= 0.000001)
                scheduler = ReduceLROnPlateau(optimizer, 'min')
                criterion = torch.nn.BCEWithLogitsLoss()
                early_stopping = TropiGAT_models.EarlyStopping(patience=60, verbose=True, path=f"{path_ensemble}/{KL_type}__{seed}.TropiGATv2.0702.pt", metric='MCC')
                try : 
                    for epoch in range(200):
                        train_loss = TropiGAT_models.train(model, graph_data_kltype, optimizer,criterion)
                        if epoch % 5 == 0:
                            # Get all metrics
                            test_loss, metrics = TropiGAT_models.evaluate(model, graph_data_kltype,criterion, graph_data_kltype["B1"].test_mask)
                            info_training_concise = f'Epoch: {epoch}\tTrain Loss: {train_loss}\tTest Loss: {test_loss}\tMCC: {metrics[3]}\tAUC: {metrics[5]}\tAccuracy: {metrics[4]}\n'
                            info_training = f'Epoch: {epoch}, Train Loss: {train_loss}, Test Loss: {test_loss},F1 Score: {metrics[0]}, Precision: {metrics[1]}, Recall: {metrics[2]}, MCC: {metrics[3]},Accuracy: {metrics[4]}, AUC: {metrics[5]}'
                            log_outfile.write(info_training_concise)
                            scheduler.step(test_loss)
                            early_stopping(metrics[3], model, epoch)
                            if early_stopping.early_stop:
                                log_outfile.write(f"Early stopping at epoch = {epoch}\n")
                                break
                    else :
                        torch.save(model, f"{path_ensemble}/{KL_type}__{seed}.TropiGATv2.0702.pt")
                    # The final eval :
                    if n == "small" : 
                        model_final = TropiGAT_models.TropiGAT_small_module(hidden_channels = 1280, heads = 1)
                    else :
                        model_final = TropiGAT_models.TropiGAT_big_module(hidden_channels = 1280, heads = 1)
                    model_final.load_state_dict(torch.load(f"{path_ensemble}/{KL_type}__{seed}.TropiGATv2.0702.pt"))
                    eval_loss, metrics = TropiGAT_models.evaluate(model_final, graph_data_kltype, criterion,graph_data_kltype["B1"].eval_mask)
                    with open(f"{path_ensemble}/Metric_Report.0702.tsv", "a+") as metric_outfile :
                        metric_outfile.write(f"{KL_type}__{seed}\t{n_prophage}\t{metrics[0]}\t{metrics[1]}\t{metrics[2]}\t{metrics[3]}\t{metrics[4]}\t{metrics[5]}\n")
                    info_eval = f'Epoch: {epoch}, F1 Score: {metrics[0]}, Precision: {metrics[1]}, Recall: {metrics[2]}, MCC: {metrics[3]},Accuracy: {metrics[4]}, AUC: {metrics[5]}'
                    log_outfile.write(f"Final evaluation ...\n{info_eval}")
                except Exception as e :
                    log_outfile.write(f"***Issue here : {e}")
                    with open(f"{path_ensemble}/Metric_Report.0702.tsv", "a+") as metric_outfile :
                        n_prophage = dico_prophage_count[KL_type]
                        metric_outfile.write(f"{KL_type}__{seed}\t{n_prophage}\t***Issue***\n")

if __name__ == '__main__':
    with ThreadPool(5) as p:
        p.map(train_graph, KLtypes)


> Move the files back 

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/ensemble_2812 \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/ensemble_2812_log_files \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023

In [ ]:

import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold

def train_graph(KL_type):
    n_splits = 3
    n_prophage = dico_prophage_count[KL_type]
    graph_data_kltype = graph_dico[KL_type]

    # Create a directory to store the models for each cross-validation step
    model_dir = f"{path_ensemble}/{KL_type}/models"
    os.makedirs(model_dir, exist_ok=True)

    # Split the data into n folds using StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_idx = 1
    for train_indices, val_indices in skf.split(graph_data_kltype["B1"].x, graph_data_kltype["B1"].y):
        print(f"Fold {fold_idx}:")
        print("Train indices:", train_indices)
        print("Validation indices:", val_indices)
        
        # Split the graph data into train and validation sets
        graph_data_train = graph_data_kltype[train_indices]
        graph_data_val = graph_data_kltype[val_indices]
        
        # Train the model for this fold
        model = TropiGAT_models.TropiGAT_small_module(hidden_channels=1280, heads=1)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.000001)
        scheduler = ReduceLROnPlateau(optimizer, 'min')
        criterion = torch.nn.BCEWithLogitsLoss()
        early_stopping = TropiGAT_models.EarlyStopping(patience=60, verbose=True, path=f"{model_dir}/fold{fold_idx}.pt", metric='MCC')
        train_loss = TropiGAT_models.train(model, graph_data_train, optimizer, criterion)
        
        # Evaluate the model on the validation set and save the metrics to a file
        test_loss, metrics = TropiGAT_models.evaluate(model, graph_data_val, criterion, graph_data_val["B1"].eval_mask)
        metrics_df = pd.DataFrame(metrics, index=[fold_idx])
        if os.path.exists(f"{model_dir}/metrics.csv"):
            metrics_df.to_csv(f"{model_dir}/metrics.csv", mode='a', header=False, index=True)
        else:
            metrics_df.to_csv(f"{model_dir}/metrics.csv", index=True)
        
        # Save the trained model for this fold
        torch.save(model, f"{model_dir}/fold{fold_idx}.pt")
        
        fold_idx += 1